In [1]:
import pandas as pd

In [2]:
import importlib

In [3]:
import pytest

In [4]:
import geopandas as gp

In [5]:
from shapely.geometry import Point

In [6]:
vrp = importlib.import_module("VRP_OR-tools_Stops_veh_tech")

In [7]:
import pickle

In [8]:
from unittest import TestCase

In [9]:
# Testing normal case where the origin and destination exist in the origin-destination travel time dataframe
def test_tt_cal_normal():
    sel_tt = pd.DataFrame()
    sel_tt['origin'] = [1000, 1000,1001,1001]
    sel_tt['destination'] = [1000,1001,1000,1001]
    sel_tt['TIME_minutes'] = [0,5,10,0]
    
    sel_dist = pd.DataFrame()
    sel_dist['Origin'] = [10, 10,12,12]
    sel_dist['Destination'] = [10,12,10,12]
    sel_dist['dist'] = [0,500,1000,0]
    
    assert vrp.tt_cal(10, 10, 1000, 1001, sel_tt, sel_dist) == 5, "incorrect travel time"

In [10]:
# Testing case where the origin and destination are not in the travel time dataframe 
# but in the orgin-destination distance dataframe
def test_tt_cal_dist():
    sel_tt = pd.DataFrame()
    sel_tt['origin'] = [1000, 1000,1001,1001]
    sel_tt['destination'] = [1000,1001,1000,1001]
    sel_tt['TIME_minutes'] = [0,5,10,0]
    
    sel_dist = pd.DataFrame()
    sel_dist['Origin'] = [10, 10,12,12]
    sel_dist['Destination'] = [10,12,10,12]
    sel_dist['dist'] = [0,500,1000,0]
    
    assert vrp.tt_cal(10, 12, 1005, 1012, sel_tt, sel_dist) == 750, "incorrect travel time"

In [11]:
# Testing case where the origin and destination are not in the travel time dataframe 
# and are not in the orgin-destination distance dataframe
def test_tt_cal_exception():
    sel_tt = pd.DataFrame()
    sel_tt['origin'] = [1000, 1000,1001,1001]
    sel_tt['destination'] = [1000,1001,1000,1001]
    sel_tt['TIME_minutes'] = [0,5,10,0]
    
    sel_dist = pd.DataFrame()
    sel_dist['Origin'] = [10, 10,12,12]
    sel_dist['Destination'] = [10,12,10,12]
    sel_dist['dist'] = [0,500,1000,0]
    
    assert vrp.tt_cal(13, 15, 1005, 1012, sel_tt, sel_dist) == 180, "incorrect travel time"

In [12]:
def test_get_geoId_normal():
    """ Testing getting geo id (census block Id) from geo Id files, case where the geo id exist in the file 
    """
    pd_df = pd.DataFrame()
    pd_df['GEOID'] = [456, 457]
    pd_df['CBPZONE'] = [1234, 1235]
    pd_df['MESOZONE'] = [101,1001]
    pd_df['geometry'] = [Point(-101.91, 33.55), Point(-98.53, 29.53)]
    
    gp_df = gp.GeoDataFrame(pd_df, geometry='geometry')
    
    assert vrp.get_geoId(101,gp_df) == 456, "incorrect geo Id"

In [13]:
def test_get_geoId_exception():
    """ Testing getting geo id (census block Id) from geo Id files, case where the geo id does not exist in the file
        This case returns -1
    """
    pd_df = pd.DataFrame()
    pd_df['GEOID'] = [456, 457]
    pd_df['CBPZONE'] = [1234, 1235]
    pd_df['MESOZONE'] = [101,1001]
    pd_df['geometry'] = [Point(-101.91, 33.55), Point(-98.53, 29.53)]
    
    gp_df = gp.GeoDataFrame(pd_df, geometry='geometry')
    
    assert vrp.get_geoId(2003,gp_df) == -1, "incorrect geo Id"

In [14]:
def test_create_data_model_delivery_external_normal():
    df_prob = pd.read_csv('test_data/df_prob_delivery.csv')
    c_prob = pd.read_csv('test_data/c_prob_delivery.csv')
    v_df = pd.read_csv('test_data/v_df_delivery.csv')
    vc_prob = pd.read_csv('test_data/vc_prob_delivery.csv')
    CBGzone_df = pd.read_csv('test_data/CBGzone_df.csv')
    tt_df = pd.read_csv('test_data/sel_tt_delivery.csv')
    dist_df = pd.read_csv('test_data/sel_dist_delivery.csv')
    prob_type = 'delivery'
    carr_id = 'B2C_2879885.0'
    depot_loc = c_prob.loc[c_prob['carrier_id'] == carr_id]['depot_zone'].values[0]
    veh = 'md_D_Diesel'
    comm = 5
    index = 'external'
    path_stops = ''  # We do not info on stop distribution for external tours
    
    data = {}
    with open('test_data/b2c_delivery_external.pickle', 'rb') as handle:
        data = pickle.load(handle)
    
    ret_data = vrp.create_data_model(df_prob, depot_loc, prob_type, v_df, vc_prob, c_prob, carr_id,
                                                CBGzone_df, tt_df, dist_df, veh, comm, index, path_stops)
    
    TestCase().assertDictEqual(ret_data, data, "incorrect data dictionary created")
    

In [15]:
def test_create_data_model_pickup_delivery_external_normal():
    df_prob = pd.read_csv('test_data/df_prob_pickup_delivery.csv')
    c_prob = pd.read_csv('test_data/c_prob_pickup_delivery.csv')
    v_df = pd.read_csv('test_data/v_df_pickup_delivery.csv')
    vc_prob = pd.read_csv('test_data/vc_prob_pickup_delivery.csv')
    CBGzone_df = pd.read_csv('test_data/CBGzone_df.csv')
    tt_df = pd.read_csv('test_data/sel_tt_pickup_delivery.csv')
    dist_df = pd.read_csv('test_data/sel_dist_pickup_delivery.csv')
    prob_type = 'pickup_delivery'
    carr_id = 'B2B_2627740_0hdt_D'
    depot_loc = c_prob.loc[c_prob['carrier_id'] == carr_id]['depot_zone'].values[0]
    veh = 'hdt_D_Diesel'
    comm = 2
    index = 'external'
    path_stops = ''  # We do not info on stop distribution for external tours
    
    data = {}
    with open('test_data/b2b_pickup_delivery_external.pickle', 'rb') as handle:
        data = pickle.load(handle)
    
    ret_data = vrp.create_data_model(df_prob, depot_loc, prob_type, v_df, vc_prob, c_prob, carr_id,
                                                CBGzone_df, tt_df, dist_df, veh, comm, index, path_stops)
    
    TestCase().assertDictEqual(ret_data, data, "incorrect data dictionary created")

In [16]:
test_create_data_model_pickup_delivery_external_normal()

veh_capacity:  36000  num_veh:  8


In [17]:
def test_create_data_model_delivery_internal_normal():
    df_prob = pd.read_csv('test_data/df_prob_delivery.csv')
    c_prob = pd.read_csv('test_data/c_prob_delivery.csv')
    v_df = pd.read_csv('test_data/v_df_delivery.csv')
    vc_prob = pd.read_csv('test_data/vc_prob_delivery.csv')
    CBGzone_df = pd.read_csv('test_data/CBGzone_df.csv')
    tt_df = pd.read_csv('test_data/sel_tt_delivery.csv')
    dist_df = pd.read_csv('test_data/sel_dist_delivery.csv')
    prob_type = 'delivery'
    carr_id = 'B2C_2879885.0'
    depot_loc = c_prob.loc[c_prob['carrier_id'] == carr_id]['depot_zone'].values[0]
    veh = 'md_D_Diesel'
    comm = 5
    index = 'internal'
    path_stops = '../../../FRISM_input_output_AT/Survey_Data/'  # We do not info on stop distribution for external tours
    
    data = {}
    with open('test_data/b2c_delivery_internal.pickle', 'rb') as handle:
        data = pickle.load(handle)
    
    ret_data = vrp.create_data_model(df_prob, depot_loc, prob_type, v_df, vc_prob, c_prob, carr_id,
                                                CBGzone_df, tt_df, dist_df, veh, comm, index, path_stops)
    
    TestCase().assertDictEqual(ret_data, data, "incorrect data dictionary created")

In [18]:
def test_create_data_model_pickup_delivery_internal_normal():
    df_prob = pd.read_csv('test_data/df_prob_pickup_delivery.csv')
    c_prob = pd.read_csv('test_data/c_prob_pickup_delivery.csv')
    v_df = pd.read_csv('test_data/v_df_pickup_delivery.csv')
    vc_prob = pd.read_csv('test_data/vc_prob_pickup_delivery.csv')
    CBGzone_df = pd.read_csv('test_data/CBGzone_df.csv')
    tt_df = pd.read_csv('test_data/sel_tt_pickup_delivery.csv')
    dist_df = pd.read_csv('test_data/sel_dist_pickup_delivery.csv')
    prob_type = 'pickup_delivery'
    carr_id = 'B2B_2627740_0hdt_D'
    depot_loc = c_prob.loc[c_prob['carrier_id'] == carr_id]['depot_zone'].values[0]
    veh = 'hdt_D_Diesel'
    comm = 5
    index = 'internal'
    path_stops = '../../../FRISM_input_output_AT/Survey_Data/'  # We do not info on stop distribution for external tours
    
    data = {}
    with open('test_data/b2b_pickup_delivery_internal.pickle', 'rb') as handle:
        data = pickle.load(handle)
    
    ret_data = vrp.create_data_model(df_prob, depot_loc, prob_type, v_df, vc_prob, c_prob, carr_id,
                                                CBGzone_df, tt_df, dist_df, veh, comm, index, path_stops)
    
    TestCase().assertDictEqual(ret_data, data, "incorrect data dictionary created")

In [11]:
def test_input_files_processing_delivery_normal():
    travel_file = 'test_data/sel_tt_delivery.csv.gz'
    dist_file = 'test_data/sel_dist_delivery.csv'
    CBGzone_file = 'test_data/CBGzone_df.geojson'
    carrier_file = 'test_data/c_prob_delivery.csv'
    payload_file = 'test_data/df_prob_delivery.csv'
    vehicleType_file = 'test_data/v_df_delivery.csv'
    
    p_df = pd.read_csv(payload_file)
    c_df = pd.read_csv(carrier_file)
    v_df = pd.read_csv(vehicleType_file)
    vc_df = pd.read_csv('test_data/vc_df_processing_delivery.csv')
    CBGzone_df = gp.read_file(CBGzone_file)
    tt_df = pd.read_csv(travel_file, compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)
    dist_df = pd.read_csv(dist_file)
    
    ret_tt_df, ret_dist_df, ret_CBGzone_df, ret_c_df, ret_p_df, ret_v_df, ret_vc_df = vrp.input_files_processing(travel_file, dist_file, 
                                                                                                                 CBGzone_file, carrier_file, payload_file, vehicleType_file)
    
    assert ret_tt_df.equals(tt_df), "incorrect travel time dataframe"
    assert ret_dist_df.equals(dist_df), "incorrect distance dataframe"
    assert ret_CBGzone_df.equals(CBGzone_df), "incorrect census block id file"
    assert ret_c_df.equals(c_df), "incorrect carrier dataframe"
    assert ret_p_df.equals(p_df), "incorrect payload dataframe"
    assert ret_v_df.equals(v_df), "incorrect vehicle dataframe"
    assert ret_vc_df.equals(vc_df), "incorrect vehicle details dataframe"

In [18]:
def test_input_files_processing_exception():
    travel_file = 'test_data/sel_tt_delivery.csv'
    dist_file = 'test_data/sel_dist_delivery.csv'
    CBGzone_file = 'test_data/CBGzone_df.geojson'
    carrier_file = 'test_data/c_prob_delivery.csv'
    payload_file = 'test_data/df_prob_delivery.csv'
    vehicleType_file = 'test_data/v_df_delivery.csv'
    
    # Checking if we capture exception from travel_file
    assert vrp.input_files_processing(travel_file, dist_file, CBGzone_file, 
                                       carrier_file, payload_file, vehicleType_file) == None, 'Missed exception on reading travel time input file format'
    #Checking if we capture exception from dist_file
    travel_file = 'test_data/sel_tt_delivery.csv.gz'
    dist_file = 'test_data/sel_dist_delivery.txt'
    assert vrp.input_files_processing(travel_file, dist_file, CBGzone_file, 
                                       carrier_file, payload_file, vehicleType_file) == None, 'Missed exception on reading distance input file format'
    #Checking if we capture exception from geoid file 
    dist_file = 'test_data/sel_dist_delivery.csv'
    CBGzone_file = 'test_data/CBGzone_df.csv'
    assert vrp.input_files_processing(travel_file, dist_file, CBGzone_file, 
                                       carrier_file, payload_file, vehicleType_file) == None, 'Missed exception on reading geo id input file format'
    # Checking if we capture exception from carrier file
    CBGzone_file = 'test_data/CBGzone_df.geojson'
    carrier_file = 'src/test_data/c_prob_delivery.csv'
    assert vrp.input_files_processing(travel_file, dist_file, CBGzone_file, 
                                       carrier_file, payload_file, vehicleType_file) == None, 'Missed exception on reading carrier input file format'
    
    # Checking if we capture exception from payload file
    carrier_file = 'test_data/c_prob_delivery.csv'
    payload_file = 'test_data/df_prob_delivery.txt'
    assert vrp.input_files_processing(travel_file, dist_file, CBGzone_file, 
                                       carrier_file, payload_file, vehicleType_file) == None, 'Missed exception on reading payload input file format'
    
    # Checking if we capture exception from payload file
    payload_file = 'test_data/df_prob_delivery.csv'
    vehicleType_file = 'test_data/v_df_delivery.txt'
    assert vrp.input_files_processing(travel_file, dist_file, CBGzone_file, 
                                       carrier_file, payload_file, vehicleType_file) == None, 'Missed exception on reading vehicle input file format'

In [19]:
test_input_files_processing_exception()

Exception:  Travel time file Not a gzipped file (b'Un')
Exception:   [Errno 2] No such file or directory: 'test_data/sel_dist_delivery.txt'
Exception:   maximum recursion depth exceeded in comparison
Exception:   [Errno 2] No such file or directory: 'src/test_data/c_prob_delivery.csv'
Exception:   [Errno 2] No such file or directory: 'test_data/df_prob_delivery.txt'
Exception:   [Errno 2] No such file or directory: 'test_data/v_df_delivery.txt'


In [15]:
test_input_files_processing_geozone_exception()

Exception:   maximum recursion depth exceeded in comparison


In [19]:
test_create_data_model_delivery_external_normal()
test_create_data_model_pickup_delivery_external_normal()
test_create_data_model_delivery_internal_normal()
test_create_data_model_pickup_delivery_internal_normal()

veh_capacity:  8000  num_veh:  12
veh_capacity:  36000  num_veh:  8
pickled data:
veh_capacity:  8000  num_veh:  12
veh_capacity:  36000  num_veh:  8
